In [1]:
pip install git+https://github.com/jqmviegas/jqm_cvi.git

  Cloning https://github.com/jqmviegas/jqm_cvi.git to /private/var/folders/t4/1zgf4p9x29d9pxmkw5z_x0100000gn/T/pip-req-build-fd7bl8jj
  Running command git clone --filter=blob:none --quiet https://github.com/jqmviegas/jqm_cvi.git /private/var/folders/t4/1zgf4p9x29d9pxmkw5z_x0100000gn/T/pip-req-build-fd7bl8jj
  Resolved https://github.com/jqmviegas/jqm_cvi.git to commit 9e1137d4f4df70c1287011e1a544b0c3fe36af36
  Preparing metadata (setup.py) ... done
  Created wheel for jqmcvi: filename=jqmcvi-1.0-cp39-cp39-macosx_10_9_x86_64.whl size=87048 sha256=6f8710bacabe2b63e323ac27d267f5a1161d4eb2a005c2fec051a95f27006f26
  Stored in directory: /private/var/folders/t4/1zgf4p9x29d9pxmkw5z_x0100000gn/T/pip-ephem-wheel-cache-ps380lxa/wheels/8c/53/48/fa640c05477263e265eaf1c7fd337aeef8ba6e74b60ffd00bd
Successfully built jqmcvi
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Application of Time2Feat on real dataset
import numpy as np
import pandas as pd
from t2f.extraction.extractor import feature_extraction
from t2f.utils.importance_old import feature_selection
from t2f.model.clustering import ClusterWrapper
from t2f.data.dataset import read_ucr_datasets
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_mutual_info_score


files=['data/BasicMotions/BasicMotions_TRAIN.txt', 'data/BasicMotions/BasicMotions_TEST.txt']
transform_type='minmax'
model_type='Hierarchical'
train_size=0.3
batch_size=500
p=4

# Read original dataset
print('Read ucr datasets: ', files)
ts_list, y_true = read_ucr_datasets(paths=files)
n_clusters = len(set(y_true))  # Get number of clusters to find

# Create cluster model
model = ClusterWrapper(n_clusters=n_clusters, model_type=model_type, transform_type=transform_type)
print('Dataset shape: {}, Num of clusters: {}'.format(ts_list.shape, n_clusters))


# This side is dedicated for the Semi-Supervised  
labels = {}
if train_size > 0:
    # Extract a subset of labelled mts to train the semi-supervised model
    idx_train, _, y_train, _ = train_test_split(np.arange(len(ts_list)), y_true, train_size=train_size)
    labels = {i: j for i, j in zip(idx_train, y_train)}
    print('Number of Labels: {}'.format(len(labels)))


Read ucr datasets:  ['data/BasicMotions/BasicMotions_TRAIN.txt', 'data/BasicMotions/BasicMotions_TEST.txt']
Dataset shape: (80, 100, 6), Num of clusters: 4
Number of Labels: 24


## Proposed RL Formulation:

Objective is to select the n features that provide the best possible time-series clustering

- State: currently selected features
- Action: select a new feature
- Reward: let $m$ be a selected metric, reward is gain in that metric
$$r_t = m_{t+1} - m_t$$
- Episode ends when current state has n features

In [2]:
df_all_feats = feature_extraction(ts_list, batch_size, p)

Feature Extraction: 100%|██████████| 480/480 [00:17<00:00, 26.84it/s]


In [3]:
from t2f.selection.selection import cleaning
df_all_feats = cleaning(df_all_feats)

In [4]:
df_all_feats

,single__0__variance_larger_than_standard_deviation,single__0__has_duplicate_max,single__0__has_duplicate_min,single__0__sum_values,single__0__abs_energy,single__0__mean_abs_change,single__0__mean_change,single__0__mean_second_derivative_central,single__0__median,single__0__mean,...,pair__euclidean__3__5,pair__minkowski__3__5,pair__braycurtis__4__5,pair__canberra__4__5,pair__chebyshev__4__5,pair__cityblock__4__5,pair__correlation__4__5,pair__cosine__4__5,pair__euclidean__4__5,pair__minkowski__4__5
0,0.0,0.0,0.0,-8.618429,10.629914,0.164450,-0.002871,-0.000061,-0.164268,-0.086184,...,4.514718,4.514718,0.795626,71.998963,2.010846,31.395836,0.681584,0.695189,4.897790,4.897790
1,0.0,0.0,1.0,-14.061870,48.609672,0.211628,-0.007186,0.000000,-0.287704,-0.140619,...,5.996254,5.996254,1.019548,79.432037,4.578337,30.005561,1.296027,1.292361,7.113090,7.113090
2,0.0,0.0,0.0,-7.982351,10.309371,0.239052,0.002724,0.000000,-0.056578,-0.079824,...,4.073435,4.073435,1.776442,93.251353,1.265102,55.512677,1.837035,1.837516,6.517196,6.517196
3,0.0,0.0,0.0,-13.702903,6.150112,0.183618,-0.005907,-0.001008,-0.139846,-0.137029,...,3.303360,3.303360,2.182169,94.029656,0.937507,30.117417,1.735502,1.732719,3.629707,3.629707
4,0.0,1.0,0.0,-26.674802,8.912128,0.111928,-0.002230,-0.000930,-0.272428,-0.266748,...,1.358499,1.358499,1.179158,85.996102,0.460764,15.671286,1.353467,1.328027,1.852333,1.852333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1.0,0.0,0.0,465.601671,6681.979256,4.081376,0.040581,0.015025,2.719462,4.656017,...,41.015942,41.015942,1.059686,76.938412,19.772877,292.893735,1.002140,1.003352,43.106176,43.106176
76,1.0,0.0,0.0,412.625960,5716.535296,3.523823,0.022801,0.000000,2.097393,4.126260,...,32.841336,32.841336,1.085719,75.991708,14.150498,293.152094,1.049881,1.048059,41.240254,41.240254
77,1.0,0.0,0.0,362.222132,4714.701692,2.839722,0.077807,0.046355,1.367126,3.622221,...,35.485859,35.485859,0.975175,72.506684,22.300418,287.923894,1.134946,1.127056,49.223002,49.223002
78,1.0,0.0,0.0,250.599170,4083.098033,2.910435,0.062136,0.000000,0.733762,2.505992,...,40.431598,40.431598,1.016970,67.432317,10.749372,251.550207,0.956278,0.952476,38.522464,38.522464


In [7]:
import random
from tsfresh.utilities.dataframe_functions import impute
import gym
from gym import spaces

In [8]:
impute(df_all_feats)

,single__0__variance_larger_than_standard_deviation,single__0__has_duplicate_max,single__0__has_duplicate_min,single__0__sum_values,single__0__abs_energy,single__0__mean_abs_change,single__0__mean_change,single__0__mean_second_derivative_central,single__0__median,single__0__mean,...,pair__euclidean__3__5,pair__minkowski__3__5,pair__braycurtis__4__5,pair__canberra__4__5,pair__chebyshev__4__5,pair__cityblock__4__5,pair__correlation__4__5,pair__cosine__4__5,pair__euclidean__4__5,pair__minkowski__4__5
0,0.0,0.0,0.0,-8.618429,10.629914,0.164450,-0.002871,-0.000061,-0.164268,-0.086184,...,4.514718,4.514718,0.795626,71.998963,2.010846,31.395836,0.681584,0.695189,4.897790,4.897790
1,0.0,0.0,1.0,-14.061870,48.609672,0.211628,-0.007186,0.000000,-0.287704,-0.140619,...,5.996254,5.996254,1.019548,79.432037,4.578337,30.005561,1.296027,1.292361,7.113090,7.113090
2,0.0,0.0,0.0,-7.982351,10.309371,0.239052,0.002724,0.000000,-0.056578,-0.079824,...,4.073435,4.073435,1.776442,93.251353,1.265102,55.512677,1.837035,1.837516,6.517196,6.517196
3,0.0,0.0,0.0,-13.702903,6.150112,0.183618,-0.005907,-0.001008,-0.139846,-0.137029,...,3.303360,3.303360,2.182169,94.029656,0.937507,30.117417,1.735502,1.732719,3.629707,3.629707
4,0.0,1.0,0.0,-26.674802,8.912128,0.111928,-0.002230,-0.000930,-0.272428,-0.266748,...,1.358499,1.358499,1.179158,85.996102,0.460764,15.671286,1.353467,1.328027,1.852333,1.852333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1.0,0.0,0.0,465.601671,6681.979256,4.081376,0.040581,0.015025,2.719462,4.656017,...,41.015942,41.015942,1.059686,76.938412,19.772877,292.893735,1.002140,1.003352,43.106176,43.106176
76,1.0,0.0,0.0,412.625960,5716.535296,3.523823,0.022801,0.000000,2.097393,4.126260,...,32.841336,32.841336,1.085719,75.991708,14.150498,293.152094,1.049881,1.048059,41.240254,41.240254
77,1.0,0.0,0.0,362.222132,4714.701692,2.839722,0.077807,0.046355,1.367126,3.622221,...,35.485859,35.485859,0.975175,72.506684,22.300418,287.923894,1.134946,1.127056,49.223002,49.223002
78,1.0,0.0,0.0,250.599170,4083.098033,2.910435,0.062136,0.000000,0.733762,2.505992,...,40.431598,40.431598,1.016970,67.432317,10.749372,251.550207,0.956278,0.952476,38.522464,38.522464


## Normalization:

The objective is to normalize the main metrics between 0 and 1. Where 0 means low quality cluster and 1 means high quality clustering.

I've adopted the Min-Max normalization:
$$score = \frac{scoreMetric - min(boundaryMetrics)}{max(boundaryMetrics) - min(boundaryMetrics)} $$

For example, for the Silhouette Score, I have:
$$score = \frac{scoreSiljouette - (-1)}{1 - (-1)} $$

For the other three metrics that doesn't have the upper limit[1], I fix a maximum limit of 1000. This means that, for calinski_harabasz_score and dunn_fast, the cluster is perfect when they reaches 1000. This limit can be adapted based on the results we obtain.

Based on the fact that the davies_bouldin_score get 0 when the cluster is perfect, the maximum value reachable is 0 and the minimum is the upperlimit. Therefore, I've applied the MinMax normalization based on these values and then I've subtructed 1 to respect the rules presented before.

[1] https://www.notion.so/Clusters-Reward-48bcb5b891b9476eb444bcc54d6874b3

In [14]:
import sys
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
from jqmcvi.base import dunn_fast


upperLimit = 1000


def normalized_DBS(df_feat_all, y_pred):
    dbs = davies_bouldin_score(df_feat_all, y_pred)
    print('dbs:', dbs)
    if dbs > upperLimit:
        dbs = upperLimit
    score = ((dbs - upperLimit)/(0-upperLimit))
    print('DBS Score: ' + str(score))
    return score

def normalized_CHS(df_feat_all, y_pred): 
    chs = calinski_harabasz_score(df_feat_all, y_pred)
    if chs > upperLimit:
        chs = upperLimit
    score = chs/upperLimit
    print('CHS Score: ' + str(score))
    return score


def normalized_DuS(df_feat_all, y_pred): 
    dus = dunn_fast(df_feat_all, y_pred)
    if dus > upperLimit:
        dus = upperLimit
    score = dus/upperLimit
    print('DuS Score: ' + str(score))
    return score

def normalized_SilS(df_feat_all, y_pred): 
    sils = silhouette_score(df_feat_all, y_pred)
    score = (sils + 1)/(2)
    print('SilS Score: ' + str(score))
    return score    

In [15]:
def obtain_score(df_feat_all, y_pred):
    scoreDBS = normalized_DBS(df_feat_all, y_pred)
    scoreCHS = normalized_CHS(df_feat_all, y_pred)
    scoreDuS = normalized_DuS(df_feat_all, y_pred)
    scoreSilS = normalized_SilS(df_feat_all, y_pred)
    scores = [scoreDBS, scoreCHS, scoreDuS, scoreSilS]
    return np.mean(scores)

In [16]:
class EpsilonGreedy:
    """Epsilon Greedy Exploration Strategy."""

    def __init__(self, initial_epsilon=1.0, min_epsilon=0.0, decay=0.99):
        """Initialize Epsilon Greedy Exploration Strategy."""
        self.initial_epsilon = initial_epsilon
        self.epsilon = initial_epsilon
        self.min_epsilon = min_epsilon
        self.decay = decay

    def choose(self, q_table, state, action_space, legal_actions):
        """Choose action based on epsilon greedy strategy."""
        if np.random.rand() < self.epsilon:
            action = random.choice(legal_actions)
        else:
            legal_actions_index = np.argmax([q_table[state][legal] for legal in legal_actions])
            action = legal_actions[legal_actions_index]

        self.epsilon = max(self.epsilon * self.decay, self.min_epsilon)
        # print(self.epsilon)
        return action

    def reset(self):
        """Reset epsilon to initial value."""
        self.epsilon = self.initial_epsilon


class QLAgent:
    """Q-learning Agent class."""

    def __init__(self, starting_state, state_space, action_space, alpha=0.5, gamma=0.95, exploration_strategy=EpsilonGreedy()):
        """Initialize Q-learning agent."""
        self.state = starting_state
        self.state_space = state_space
        self.action_space = action_space
        self.action = None
        self.alpha = alpha
        self.gamma = gamma
        self.q_table = {self.state: [0 for _ in range(action_space.n)]}
        self.exploration = exploration_strategy
        self.acc_reward = 0

    def act(self, legal_actions):
        """Choose action based on Q-table."""
        self.action = self.exploration.choose(self.q_table, self.state, self.action_space, legal_actions)
        return self.action

    def learn(self, next_state, reward, done=False):
        """Update Q-table with new experience."""
        if next_state not in self.q_table:
            self.q_table[next_state] = [0 for _ in range(self.action_space.n)]

        s = self.state
        s1 = next_state
        a = self.action
        self.q_table[s][a] = self.q_table[s][a] + self.alpha * (
            reward + self.gamma * max(self.q_table[s1]) - self.q_table[s][a]
        )
        self.state = s1
        self.acc_reward += reward

In [17]:
class FeatureSelectionEnvironment(gym.Env):

    def __init__(self, df_features, n_features, arr, clustering_model) -> None:
        self.observation_space = spaces.Box(0, 1, shape=(len(df_features.columns),), dtype=np.float32)
        self.action_space = spaces.Discrete(len(df_features.columns))
        
        self.all_features = df_features.copy()
        self.current_state = np.zeros(len(self.all_features.columns))
        self.n_features = n_features
        self.clustering_model = clustering_model
        
        self.past_reward = 0

    
    def _get_obs(self) -> np.array:
        return self.current_state

    def _get_info(self) -> dict:
        return {
            'legal_actions': [action for action in range(self.action_space.n) if not self.current_state[action]]
        }
    
    def reset(self, seed=None, options=None):
        self.current_state = np.zeros(len(self.all_features.columns))
        observation = self._get_obs()
        info = self._get_info()
        return observation, info
    
    def _get_reward(self, action):
        selected_features = [feature for i, feature in enumerate(self.all_features.columns) if env.current_state[i]]
        y_pred = self.clustering_model.fit_predict(self.all_features[selected_features])
        
        try:
            """
            Test to see if this works well, otherwise we have two current options:
            (1) Use only the features selected by RL to compute metrics
            (2) Use raw time series to compute metrics
            """
            score = obtain_score(df_all_feats, y_pred)
            # score = silhouette_score(ts_list, y_pred)
        except ValueError:
            # happens when all labels have same value, thus no real "clustering" has occurred
            score = -1
        
        gain = score - self.past_reward
        self.past_reward = score
        
        return gain
    
    def step(self, action):
        self.current_state[action] = 1
        reward = self._get_reward(action)
        observation = self._get_obs()
        info = self._get_info()
        
        if len(self.current_state[self.current_state == 1]) < self.n_features:
            terminated = False
        else:
            terminated = True

        return observation, reward, terminated, False, info
    
    def render(self):
        pass

    def _render_frame(self):
        pass
    
    def close(self):
        pass


In [18]:
episodes = 2
n_features = 3


env = FeatureSelectionEnvironment(
    df_features=df_all_feats,
    n_features=n_features,
    arr=ts_list,
    clustering_model=model,
)

obs, info = env.reset()
done = False

agent = QLAgent(starting_state=tuple(obs), state_space=env.observation_space, action_space=env.action_space)

for episode in range(episodes):
    print(f'Episode {episode}')
    while not done:
        action = agent.act(info['legal_actions'])
        print('Action:', action)
        next_obs, reward, terminated, truncated, info = env.step(action)
        print(reward)
        agent.learn(tuple(next_obs), reward, done)
        done = terminated or truncated
    obs, info = env.reset()
    done = False

Episode 0
Action: 13
dbs: 1.5378054164865542
DBS Score: 0.9984621945835135
CHS Score: 0.03842841673994315
DuS Score: 7.81306894355731e-05
SilS Score: 0.5387635508596669
0.39393307321813975
Action: 3222
dbs: 1.876031435632649
DBS Score: 0.9981239685643674
CHS Score: 0.1630820343041435
DuS Score: 0.00015129269026743802
SilS Score: 0.6272293293168947
0.0532135830007785
Action: 1154
dbs: 6.103249890044874
DBS Score: 0.9938967501099552
CHS Score: 0.04446595398330799
DuS Score: 9.342347413927589e-05
SilS Score: 0.500368492782494
-0.06244050113144417
Episode 1
Action: 1481
dbs: 7.186300320020237
DBS Score: 0.9928136996799798
CHS Score: 0.0020168139258715887
DuS Score: 6.76504499484022e-05
SilS Score: 0.44529163065056443
-0.024658706410883025
Action: 1719
dbs: 6.683146189497698
DBS Score: 0.9933168538105023
CHS Score: 0.0021776634941127015
DuS Score: 6.790104538891928e-05
SilS Score: 0.4583347378974689
0.00342684038527713
Action: 2013
dbs: 13.230237843360742
DBS Score: 0.9867697621566393
CHS S

In [39]:
# Focus on feature selection, with unsupervised metrics
# Let's start with hierarchical clustering